* https://www.gov.uk/guidance/mot-inspection-manual-for-private-passenger-and-light-commercial-vehicles
* https://www.data.gov.uk/dataset/e3939ef8-30c7-4ca8-9c7c-ad9475cc9b2f/anonymised-mot-tests-and-results
* https://www.gov.uk/government/news/mot-changes-20-may-2018

In [1]:
from dask.distributed import Client
from distributed import progress
from glob import glob
from operator import itemgetter
import dask.dataframe as dd
import csv
import pandas as pd

In [2]:
client = Client()
client

<Client: 'tcp://127.0.0.1:46493' processes=4 threads=12, memory=31.30 GiB>

In [3]:
txt_files = glob('data/test_*.txt')
csv_files = glob('data/unzipped/**/*.csv', recursive=True)

files = txt_files + csv_files
len(files)

104

In [4]:
def sniff_dialect(file):
    with open(file) as csvfile:
        dialect = csv.Sniffer().sniff(csvfile.read(10000))
        dialect_info = dict(dialect.__dict__)
        dialect_info['file'] = file
    return dialect_info

In [5]:
dialects = [sniff_dialect(file) for file in files]

csv_info_df = (
    pd.DataFrame(dialects)
    .drop(columns=['__module__', '_name', '__doc__'])
    .set_index('file')
    .sort_index()
    .reset_index()
)

csv_info_df

file lineterminator  quoting  doublequote   
0                data/test_item_2005.txt           \r\n        0        False  \
1                data/test_item_2006.txt           \r\n        0        False   
2                data/test_item_2007.txt           \r\n        0        False   
3                data/test_item_2008.txt           \r\n        0        False   
4                data/test_item_2009.txt           \r\n        0        False   
..                                   ...            ...      ...          ...   
99   data/unzipped/test_result_31869.csv           \r\n        0        False   
100  data/unzipped/test_result_31870.csv           \r\n        0        False   
101  data/unzipped/test_result_31871.csv           \r\n        0        False   
102  data/unzipped/test_result_31876.csv           \r\n        0        False   
103  data/unzipped/test_result_31879.csv           \r\n        0        False   

    delimiter quotechar  skipinitialspace  
0           |         "             False  
1           |         "             False  
2           |         "             False  
3           |         "             False  
4           |         "             False  
..        ...       ...               ...  
99          |         "             False  
100         |         "             False  
101         |         "             False  
102         |         "             False  
103         |         "             False  

[104 rows x 7 columns]

In [6]:
csv_info_df.lineterminator.describe()

count      104
unique       1
top       \r\n
freq       104
Name: lineterminator, dtype: object

In [7]:
csv_info_df.skipinitialspace.describe()

count       104
unique        1
top       False
freq        104
Name: skipinitialspace, dtype: object

In [8]:
csv_info_df.delimiter.unique()

array(['|', ','], dtype=object)

In [9]:
results_files_df = csv_info_df[csv_info_df.file.str.contains('result')]
results_files_df

file lineterminator   
12                           data/test_result_2005.txt           \r\n  \
13                           data/test_result_2006.txt           \r\n   
14                           data/test_result_2007.txt           \r\n   
15                           data/test_result_2008.txt           \r\n   
16                           data/test_result_2009.txt           \r\n   
17                           data/test_result_2010.txt           \r\n   
18                           data/test_result_2011.txt           \r\n   
19                           data/test_result_2012.txt           \r\n   
20                           data/test_result_2013.txt           \r\n   
21                           data/test_result_2014.txt           \r\n   
22                           data/test_result_2015.txt           \r\n   
23                           data/test_result_2016.txt           \r\n   
42   data/unzipped/dft_test_result-from-2018-01-01_...           \r\n   
43   data/unzipped/dft_test_result-from-2018-04-01_...           \r\n   
44   data/unzipped/dft_test_result-from-2018-07-01_...           \r\n   
45   data/unzipped/dft_test_result-from-2018-10-01_...           \r\n   
46   data/unzipped/dft_test_result-from-2019-01-01_...           \r\n   
47   data/unzipped/dft_test_result-from-2019-04-01_...           \r\n   
48   data/unzipped/dft_test_result-from-2019-07-01_...           \r\n   
49   data/unzipped/dft_test_result-from-2019-10-01_...           \r\n   
50   data/unzipped/dft_test_result-from-2020-01-01_...           \r\n   
51   data/unzipped/dft_test_result-from-2020-04-01_...           \r\n   
52   data/unzipped/dft_test_result-from-2020-07-01_...           \r\n   
53   data/unzipped/dft_test_result-from-2020-10-01_...           \r\n   
79                       data/unzipped/test_result.csv           \r\n   
80   data/unzipped/test_result_2022/test_result_202...           \r\n   
81   data/unzipped/test_result_2022/test_result_202...           \r\n   
82   data/unzipped/test_result_2022/test_result_202...           \r\n   
83   data/unzipped/test_result_2022/test_result_202...           \r\n   
84   data/unzipped/test_result_2022/test_result_202...           \r\n   
85   data/unzipped/test_result_2022/test_result_202...           \r\n   
86   data/unzipped/test_result_2022/test_result_202...           \r\n   
87   data/unzipped/test_result_2022/test_result_202...           \r\n   
88   data/unzipped/test_result_2022/test_result_202...           \r\n   
89   data/unzipped/test_result_2022/test_result_202...           \r\n   
90   data/unzipped/test_result_2022/test_result_202...           \r\n   
91   data/unzipped/test_result_2022/test_result_202...           \r\n   
92                 data/unzipped/test_result_31859.csv           \r\n   
93                 data/unzipped/test_result_31860.csv           \r\n   
94                 data/unzipped/test_result_31861.csv           \r\n   
95                 data/unzipped/test_result_31862.csv           \r\n   
96                 data/unzipped/test_result_31863.csv           \r\n   
97                 data/unzipped/test_result_31864.csv           \r\n   
98                 data/unzipped/test_result_31868.csv           \r\n   
99                 data/unzipped/test_result_31869.csv           \r\n   
100                data/unzipped/test_result_31870.csv           \r\n   
101                data/unzipped/test_result_31871.csv           \r\n   
102                data/unzipped/test_result_31876.csv           \r\n   
103                data/unzipped/test_result_31879.csv           \r\n   

     quoting  doublequote delimiter quotechar  skipinitialspace  
12         0        False         |         "             False  
13         0        False         |         "             False  
14         0        False         |         "             False  
15         0        False         |         "             False  
16         0        False         |         "             False  
17 

## Results

In [10]:
def read_results(file, sep):
    df = dd.read_csv(file,
                     delimiter=sep,
                     doublequote=False,
                     on_bad_lines='warn',
                     include_path_column=True,
                     parse_dates=['test_date'],
                     dtype_backend='pyarrow',
                     # engine='pyarrow',
                    )
    return df

def parse_dates_pd(df):
    df['first_use_date'] = pd.to_datetime(df['first_use_date'], format='ISO8601', errors='coerce')
    return df

def parse_dates_dd(df):
    df['first_use_date'] = dd.to_datetime(df['first_use_date'], format='ISO8601', errors='coerce')
    return df

In [11]:
result_records = results_files_df[['file', 'delimiter']].to_records(index=False)
ddfs = [read_results(file, sep) for file, sep in result_records]

In [12]:
ddf = (
    dd.concat(ddfs)
    #.map_partitions(parse_dates_pd) # pandas version
    .pipe(parse_dates_dd) # dask version
)

In [13]:
ddf.dtypes

test_id               int64[pyarrow]
vehicle_id            int64[pyarrow]
test_date             datetime64[ns]
test_class_id         int64[pyarrow]
test_type            string[pyarrow]
test_result          string[pyarrow]
test_mileage          int64[pyarrow]
postcode_area        string[pyarrow]
make                 string[pyarrow]
model                string[pyarrow]
colour               string[pyarrow]
fuel_type            string[pyarrow]
cylinder_capacity     int64[pyarrow]
first_use_date        datetime64[ns]
path                        category
dtype: object

Task exception was never retrieved
future: <Task finished name='Task-3315' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-3483' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-3405' cor

Task exception was never retrieved
future: <Task finished name='Task-3312' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-3333' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-3469' cor

Task exception was never retrieved
future: <Task finished name='Task-3347' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-3194' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-3438' cor

Task exception was never retrieved
future: <Task finished name='Task-2438' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-2459' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-2527' cor

...

Task exception was never retrieved
future: <Task finished name='Task-2193' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-2232' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-2284' cor

Task exception was never retrieved
future: <Task finished name='Task-1993' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-2049' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-2156' cor

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_93725/1354886235.py", line 1, in <module>
    ddf.info(verbose=True)
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/dask/dataframe/core.py", line 6054, in info
    zip(computations.keys(), da.compute(*computations.values()))
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/dask/base.py", line 599, in compute
    results = schedule(dsk, keys, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 3226, in get
    results = self.gather(packed, asynchronous=asynchronous, direct=direct)
              

Task exception was never retrieved
future: <Task finished name='Task-2925' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-2921' coro=<Client._gather.<locals>.wait() done, defined at /home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py:2189> exception=AllExit()>
Traceback (most recent call last):
  File "/home/giles/.miniconda3/envs/pydatalondon2023/lib/python3.11/site-packages/distributed/client.py", line 2198, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-2915' cor

In [14]:
ddf.head()

test_id  vehicle_id  test_date  test_class_id test_type test_result   
0   804664368   256274986 2005-01-01              0        NT           P  \
1   392603376   633988704 2005-01-01              0        NT           P   
2  1894843206  1320781748 2005-01-01              0        NT           P   
3   830908928  1263031090 2005-01-01              4        NT           P   
4   727535460  1123257842 2005-01-01              4        NT           P   

   test_mileage postcode_area        make         model  colour fuel_type   
0         23459            TF        FORD  UNCLASSIFIED  SILVER        PE  \
1         40961             E       LOTUS  UNCLASSIFIED     RED        PE   
2         16416             S    VAUXHALL  UNCLASSIFIED    BLUE        PE   
3         93318             W  LAND ROVER   109 V8 S.W.    BLUE        PE   
4        121930            RG     CITROEN            AX   WHITE        DI   

   cylinder_capacity first_use_date   
0               <NA>            NaT  \
1               <NA>            NaT   
2               <NA>            NaT   
3               3528     1981-04-06   
4               1360     1993-08-31   

                                                path  
0  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
1  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
2  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
3  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
4  /home/giles/Projects/DS/PyData/mot_pandas2_pol...

In [15]:
f = ddf.to_parquet('test_result.parquet', write_index=False, overwrite=True, compute=False).persist()
progress(f)

VBox()

Skipping line 115394: expected 14 fields, saw 16

Skipping line 281444: expected 14 fields, saw 16

Skipping line 46202: expected 14 fields, saw 16

Skipping line 86979: expected 14 fields, saw 16

Skipping line 361415: expected 14 fields, saw 16

Skipping line 566945: expected 14 fields, saw 16



### Roundtrip

In [20]:
ddf_result = dd.read_parquet('test_result.parquet')

In [17]:
# Note the differing dtypes
ddf_result.info(verbose=True)

<class 'dask.dataframe.core.DataFrame'>
Index: 639506962 entries, 0 to 791798
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   test_id            639506962 non-null      int64[pyarrow]
 1   vehicle_id         639506962 non-null      int64[pyarrow]
 2   test_date          639506962 non-null      datetime64[ns]
 3   test_class_id      639506962 non-null      int64[pyarrow]
 4   test_type          639506962 non-null      string
 5   test_result        639506962 non-null      string
 6   test_mileage       632574944 non-null      int64[pyarrow]
 7   postcode_area      639506962 non-null      string
 8   make               639506927 non-null      string
 9   model              639505681 non-null      string
10   colour             639506962 non-null      string
11   fuel_type          639506962 non-null      string
12   cylinder_capacity  638550168 non-null      int64[pyarrow]
13   first_use_date     639479303 

In [21]:
ddf_result.head()

test_id  vehicle_id  test_date  test_class_id test_type test_result   
0   804664368   256274986 2005-01-01              0        NT           P  \
1   392603376   633988704 2005-01-01              0        NT           P   
2  1894843206  1320781748 2005-01-01              0        NT           P   
3   830908928  1263031090 2005-01-01              4        NT           P   
4   727535460  1123257842 2005-01-01              4        NT           P   

   test_mileage postcode_area        make         model  colour fuel_type   
0         23459            TF        FORD  UNCLASSIFIED  SILVER        PE  \
1         40961             E       LOTUS  UNCLASSIFIED     RED        PE   
2         16416             S    VAUXHALL  UNCLASSIFIED    BLUE        PE   
3         93318             W  LAND ROVER   109 V8 S.W.    BLUE        PE   
4        121930            RG     CITROEN            AX   WHITE        DI   

   cylinder_capacity first_use_date   
0               <NA>            NaT  \
1               <NA>            NaT   
2               <NA>            NaT   
3               3528     1981-04-06   
4               1360     1993-08-31   

                                                path  
0  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
1  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
2  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
3  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
4  /home/giles/Projects/DS/PyData/mot_pandas2_pol...

In [22]:
ddf_result.npartitions

854

In [23]:
rover_df = ddf_result.query('vehicle_id == 1238787680').compute()

In [24]:
rover_df

test_id  vehicle_id  test_date  test_class_id test_type test_result   
619621  523600018  1238787680 2005-07-29              4        NT           P  \
75461   566248504  1238787680 2006-08-11              4        NT           P   
738312  608896990  1238787680 2007-07-13              4        NT           P   
96799   651545476  1238787680 2008-07-17              4        NT           P   
163440  694193962  1238787680 2009-07-22              4        NT           F   
594408  736842448  1238787680 2009-08-03              4        RT           P   
72562   779490934  1238787680 2010-07-15              4        NT           P   
50693   822139420  1238787680 2011-07-11              4        NT           P   
562174  864787906  1238787680 2012-07-17              4        NT           F   
282107  907436392  1238787680 2012-07-20              4        RT           P   
29791   950084878  1238787680 2013-07-22              4        NT           P   
269203  318284506  1238787680 2014-07-30              4        NT           F   
463056  360932992  1238787680 2014-08-08              4        RT           P   
495617  596594508  1238787680 2015-08-03              4        NT           P   
46354   846334843  1238787680 2016-08-16              4        NT           P   

        test_mileage postcode_area   make model colour fuel_type   
619621         62931            CV  ROVER   200    RED        PE  \
75461          73606            CH  ROVER   200    RED        PE   
738312         83496            CH  ROVER   200    RED        PE   
96799          96680            CH  ROVER   200    RED        PE   
163440        103333            NG  ROVER   200    RED        PE   
594408        103575            NG  ROVER   200    RED        PE   
72562         115188            LU  ROVER   200    RED        PE   
50693         126169            LU  ROVER   200    RED        PE   
562174        205404            LU  ROVER   200    RED        PE   
282107        138108            LU  ROVER   200    RED        PE   
29791         147547            CH  ROVER   200    RED        PE   
269203        154280            CH  ROVER   200    RED        PE   
463056        154672            CH  ROVER   200    RED        PE   
495617        163350            LU  ROVER   200    RED        PE   
46354         167933            LD  ROVER   200    RED        PE   

        cylinder_capacity first_use_date   
619621               1796     1997-09-08  \
75461                1796     1997-09-08   
738312               1796     1997-09-08   
96799                1796     1997-09-08   
163440               1796     1997-09-08   
594408               1796     1997-09-08   
72562                1796     1997-09-08   
50693                1796     1997-09-08   
562174               1796     1997-09-08   
282107               1796     1997-09-08   
29791                1796     1997-09-08   
269203               1796     1997-09-08   
463056               1796     1997-09-08   
495617               1796     1997-09-08   
46354                1796     1997-09-08   

                                                     path  
619621  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
75461   /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
738312  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
96799   /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
163440  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
594408  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
72562   /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
50693   /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
562174  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
282107  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
29791   /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
269203  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
463056  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
495617  /home/giles/Projects/DS/PyData/mot_pandas2_pol...  
46354   /home/giles

In [25]:
rover_df.set_index('test_date').test_mileage.plot(marker='.')

<Axes: xlabel='test_date'>

## Items

In [26]:
item_files_df = csv_info_df[csv_info_df.file.str.contains('item')]
item_files_df

file lineterminator  quoting   
0                             data/test_item_2005.txt           \r\n        0  \
1                             data/test_item_2006.txt           \r\n        0   
2                             data/test_item_2007.txt           \r\n        0   
3                             data/test_item_2008.txt           \r\n        0   
4                             data/test_item_2009.txt           \r\n        0   
5                             data/test_item_2010.txt           \r\n        0   
6                             data/test_item_2011.txt           \r\n        0   
7                             data/test_item_2012.txt           \r\n        0   
8                             data/test_item_2013.txt           \r\n        0   
9                             data/test_item_2014.txt           \r\n        0   
10                            data/test_item_2015.txt           \r\n        0   
11                            data/test_item_2016.txt           \r\n        0   
25                  data/unzipped/dft_item_detail.csv           \r\n        0   
30  data/unzipped/dft_test_item-from-2018-01-01_00...           \r\n        0   
31  data/unzipped/dft_test_item-from-2018-04-01_00...           \r\n        0   
32  data/unzipped/dft_test_item-from-2018-07-01_00...           \r\n        0   
33  data/unzipped/dft_test_item-from-2018-10-01_00...           \r\n        0   
34  data/unzipped/dft_test_item-from-2019-01-01_00...           \r\n        0   
35  data/unzipped/dft_test_item-from-2019-04-01_00...           \r\n        0   
36  data/unzipped/dft_test_item-from-2019-07-01_00...           \r\n        0   
37  data/unzipped/dft_test_item-from-2019-10-01_00...           \r\n        0   
38  data/unzipped/dft_test_item-from-2020-01-01_00...           \r\n        0   
39  data/unzipped/dft_test_item-from-2020-04-01_00...           \r\n        0   
40  data/unzipped/dft_test_item-from-2020-07-01_00...           \r\n        0   
41  data/unzipped/dft_test_item-from-2020-10-01_00...           \r\n        0   
54                        data/unzipped/test_item.csv           \r\n        0   
55  data/unzipped/test_item_2021/test_item_2022060...           \r\n        0   
56  data/unzipped/test_item_2021/test_item_2022060...           \r\n        0   
57  data/unzipped/test_item_2021/test_item_2022060...           \r\n        0   
58  data/unzipped/test_item_2021/test_item_2022060...           \r\n        0   
59  data/unzipped/test_item_2021/test_item_2022060...           \r\n        0   
60  data/unzipped/test_item_2021/test_item_2022060...           \r\n        0   
61  data/unzipped/test_item_2021/test_item_2022060...           \r\n        0   
62  data/unzipped/test_item_2021/test_item_2022060...           \r\n        0   
63  data/unzipped/test_item_2021/test_item_2022060...           \r\n        0   
64  data/unzipped/test_item_2021/test_item_2022060...           \r\n        0   
65  data/unzipped/test_item_2021/test_item_2022060...           \r\n        0   
66  data/unzipped/test_item_2021/test_item_2022060...           \r\n        0   
67                  data/unzipped/test_item_26597.csv           \r\n        0   
68                  data/unzipped/test_item_26598.csv           \r\n        0   
69                  data/unzipped/test_item_26599.csv           \r\n        0   
70                  data/unzipped/test_item_26600.csv           \r\n        0   
71                  data/unzipped/test_item_26601.csv           \r\n        0   
72                  data/unzipped/test_item_26603.csv           \r\n        0   
73                  data/unzipped/test_item_26606.csv           \r\n        0   
74                  data/unzipped/test_item_26607.csv           \r\n        0   
75                  data/unzipped/test_item_26609.csv           \r\n        0   
76                  data/unzipped/test_item_26611.csv           \r\n        0   
77                  data/unzipped/test_item_26612.csv           \r\n        0   

In [27]:
def read_items(file, sep):
    df = dd.read_csv(file,
                     delimiter=sep,
                     dtype={'dangerous_mark': 'string[pyarrow]'},
                     dtype_backend='pyarrow',
                     # engine='pyarrow',
                    )
        
    return df

item_records = item_files_df[['file', 'delimiter']].to_records(index=False)
item_ddfs = [read_items(file, sep) for file, sep in item_records]

In [28]:
item_ddf = dd.concat(item_ddfs)

In [29]:
item_ddf.dtypes

test_id                      int64[pyarrow]
rfr_id                       int64[pyarrow]
rfr_type_code               string[pyarrow]
location_id                  int64[pyarrow]
dangerous_mark              string[pyarrow]
test_class_id                int64[pyarrow]
test_item_id                 int64[pyarrow]
minor_item                  string[pyarrow]
rfr_desc                    string[pyarrow]
rfr_loc_marker              string[pyarrow]
rfr_insp_manual_desc        string[pyarrow]
rfr_advisory_text           string[pyarrow]
test_item_set_section_id     int64[pyarrow]
dtype: object

In [30]:
item_ddf.info()
item_ddf.head()

<class 'dask.dataframe.core.DataFrame'>
Columns: 13 entries, test_id to test_item_set_section_id
dtypes: int64[pyarrow](6), string(1), string[pyarrow](6)

test_id  rfr_id rfr_type_code  location_id dangerous_mark   
0  1343771031    8394             A            7           <NA>  \
1  1560692612    8394             F           13           <NA>   
2  1560692612    7977             F            9           <NA>   
3  1560692612    7916             A            7           <NA>   
4  1560692612    7916             A            9           <NA>   

   test_class_id  test_item_id minor_item rfr_desc rfr_loc_marker   
0           <NA>          <NA>       <NA>     <NA>           <NA>  \
1           <NA>          <NA>       <NA>     <NA>           <NA>   
2           <NA>          <NA>       <NA>     <NA>           <NA>   
3           <NA>          <NA>       <NA>     <NA>           <NA>   
4           <NA>          <NA>       <NA>     <NA>           <NA>   

  rfr_insp_manual_desc rfr_advisory_text  test_item_set_section_id  
0                 <NA>              <NA>                      <NA>  
1                 <NA>              <NA>                      <NA>  
2                 <NA>              <NA>                      <NA>  
3                 <NA>              <NA>                      <NA>  
4                 <NA>              <NA>                      <NA>

In [31]:
f = (
    item_ddf
    .to_parquet('item.parquet', write_index=False, overwrite=True, compute=False)
    .persist()
)

progress(f)

VBox()

In [33]:
item_ddf = dd.read_parquet('item.parquet')

In [34]:
rfr_counts = item_ddf.rfr_id.value_counts().compute()

In [35]:
rfr_counts

rfr_id
8394     106966369
31194     49855966
8141      45616711
8293      36441213
8310      30535430
           ...    
8574             1
8577             1
8838             1
8850             1
9716             1
Name: count, Length: 7493, dtype: int64[pyarrow]

## Reasons for rejection (test fail)

Not sure if this includes advisories

In [36]:
rfr_df = dd.read_csv('data/unzipped/dft_item_detail.csv', sep='|', dtype_backend='pyarrow')
rfr_df.info(verbose=True)
rfr_df.head()

<class 'dask.dataframe.core.DataFrame'>
RangeIndex: 20957 entries, 0 to 20956
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   rfr_id                    20957 non-null      int64[pyarrow]
 1   test_class_id             20957 non-null      int64[pyarrow]
 2   test_item_id              20957 non-null      int64[pyarrow]
 3   minor_item                20957 non-null      string[pyarrow]
 4   rfr_desc                  20957 non-null      string[pyarrow]
 5   rfr_loc_marker            20957 non-null      string[pyarrow]
 6   rfr_insp_manual_desc      20799 non-null      string[pyarrow]
 7   rfr_advisory_text         6780 non-null      string[pyarrow]
 8   test_item_set_section_id  20957 non-null      int64[pyarrow]
dtypes: int64[pyarrow](4), string[pyarrow](5)
memory usage: 3.7 MB


rfr_id  test_class_id  test_item_id minor_item   
0       4              1             3          Y  \
1       4              2             3          Y   
2       5              1             3          Y   
3       5              2             3          Y   
4       6              1             3          Y   

                             rfr_desc rfr_loc_marker   
0                             missing              N  \
1                             missing              N   
2       damaged and function impaired              N   
3       damaged and function impaired              N   
4  deteriorated and function impaired              N   

                                rfr_insp_manual_desc   
0                         an obligatory lamp missing  \
1                         an obligatory lamp missing   
2  so damaged or deteriorated that its function i...   
3  so damaged or deteriorated that its function i...   
4  so damaged or deteriorated that its function i...   

                        rfr_advisory_text  test_item_set_section_id  
0                                    <NA>                         1  
1                                    <NA>                         1  
2       damaged but function not impaired                         1  
3       damaged but function not impaired                         1  
4  deteriorated but function not impaired                         1

In [41]:
rfr_df.query('rfr_id == 8394').compute()

rfr_id  test_class_id  test_item_id minor_item   
11001    8394              3          5655          Y  \
11002    8394              4          5655          Y   
11003    8394              7          5655          Y   

                                      rfr_desc rfr_loc_marker   
11001  tread depth below requirements of 1.6mm              Y  \
11002  tread depth below requirements of 1.6mm              Y   
11003  tread depth below requirements of 1.6mm              Y   

                                    rfr_insp_manual_desc   
11001  The grooves of the tread pattern are not at le...  \
11002  The grooves of the tread pattern are not at le...   
11003  The grooves of the tread pattern are not at le...   

                   rfr_advisory_text  test_item_set_section_id  
11001  worn close to the legal limit                      5650  
11002  worn close to the legal limit                      5650  
11003  worn close to the legal limit                      5650